In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
def import_data(data,root=".",import_folder="clean_datasets"):
    def _import_on_time(dataset):
        DF = pd.read_csv(dataset,low_memory=False).drop(columns=["Unnamed: 0"])
        DF["Route"] = [tuple(l) for l in DF["Route"].str.replace("\(|\)|\'","",regex=True).str.split(", ")]
        DF["Scheduled Time"] = pd.to_datetime(DF["Scheduled Time"])
        return DF

    def _import_traffic_counts(dataset):
        DF = pd.read_csv(dataset).drop(columns="Unnamed: 0")
        DF["Timestamp"] = pd.to_datetime(DF["Timestamp"]) 
        return DF

    def _import_lane_closure(dataset):
        DF = pd.read_csv(dataset).drop(columns="Unnamed: 0")
        DF["Boundaries"] = DF["Boundaries"].str.replace("\[|\]|\'","",regex=True).str.split(", ")
        DF["Direction"] = DF["Direction"].str.replace("\[|\]|\'","",regex=True).str.split(", ")
        DF["Date Closed - From"] = pd.to_datetime(DF["Date Closed - From"]) 
        DF["Date Closed - To"] = pd.to_datetime(DF["Date Closed - To"]) 
        DF["Geometry"] = [[[(float(l.replace("(","").split(", ")[0]),float(l.replace(")","").split(", ")[1])) for l in s.split("), (")] for s in m] 
                            for m in [j.replace("[[","").replace("]]","").split("], [") for j in DF["Geometry"].values]]
        return DF

    def _import_road_network(dataset):
        DF = pd.read_csv(dataset).drop(columns="Unnamed: 0")
        DF.loc[:,"Location"] = np.array([to_2tuple_list(s) for s in DF["Location"]],dtype=object)
        return DF

    def _import_sites(dataset):
        return pd.read_csv(dataset)

    def _import_stops(dataset):
        return pd.read_csv(dataset).drop(columns="Unnamed: 0")
    
    import_dict = {
        "ON_TIME":[f"{root}/{import_folder}/ON_TIME.csv",_import_on_time],
        "TRAFFIC_COUNTS":[f"{root}/{import_folder}/TRAFFIC_COUNTS.csv",_import_traffic_counts],
        "SITES":[f"{root}/{import_folder}/SITES.csv",_import_sites],
        "STOPS":[f"{root}/{import_folder}/STOPS.csv",_import_stops],
        "LANE_CLOSURE":[f"{import_folder}/LANE_CLOSURE.csv",_import_lane_closure],
        "ROAD_NETWORK":[f"{import_folder}/ROAD_NETWORK.csv",_import_road_network],
        ## August 2021 data
        "ON_TIME_AUG_21":[f"{root}/{import_folder}/ON_TIME_AUG_21.csv",_import_on_time],
        "TRAFFIC_COUNTS_AUG_21":[f"{root}/{import_folder}/TRAFFIC_COUNTS_AUG_21.csv",_import_traffic_counts],
        "LANE_CLOSURE_AUG_21":[f"{root}/{import_folder}/LANE_CLOSURE_AUG_21.csv",_import_lane_closure],
    }
    dataset,import_func = import_dict[data]
    return import_func(dataset)

In [ ]:
def to_2tuple(s,dtype=float):
    t1,t2 = s.replace("(","").replace(")","").split(", ")
    return (dtype(t1),dtype(t2))

def to_2tuple_r(s,dtype=float):
    t1,t2 = s.replace("(","").replace(")","").split(", ")
    return (dtype(t2),dtype(t1))

def to_2tuple_list(s,rev=False,dtype=float):
    s = s.replace("[","").replace("]","").replace("), (",");(").split(";")
    if rev: return [to_2tuple_r(t,dtype) for t in s] 
    else: return [to_2tuple(t,dtype) for t in s]

In [67]:
def to_tuple(s,rev=False,dtype=float):
    t = s.strip("[(|)]").split(", ")
    if rev: t = t[::-1]
    return tuple(map(dtype,t))

def to_tuple_l(s,rev=False,dtype=float):
    t = s.strip("[(|)]").split("), (")
    return [to_tuple(i,rev,dtype) for i in t]

# to_tuple("(71, Arlington, Portage via Sinclair)",dtype=str)
to_tuple_l("[(49.810557383041, -97.143794801994), (49.810501821987, -97.143939770317)]",dtype=float)

[(49.810557383041, -97.143794801994), (49.810501821987, -97.143939770317)]

In [39]:
ROAD_NETWORK = import_data("ROAD_NETWORK")
ROAD_NETWORK

,Block ID,Street Name,Number of Lanes,Oneway,Speed Limit,Location,Num_Points
0,1873.0,Sifton,2.0,0,50,"[(49.810557383041, -97.143794801994), (49.8105...",2
1,23974.0,Valde,2.0,0,50,"[(49.910940712709, -97.003620088172), (49.9109...",2
2,23779.0,Lagimodiere,2.0,1,80,"[(49.8671557845, -97.066599484474), (49.869726...",2
3,70342.0,Sage Creek,2.0,2,50,"[(49.833747499855, -97.049568380871), (49.8337...",2
4,24031.0,Point,2.0,0,50,"[(49.846976634414, -97.145356293446), (49.8472...",2
...,...,...,...,...,...,...,...
28957,17669.0,Newton,2.0,0,50,"[(49.943945001092, -97.11556803066), (49.94406...",21
28958,23040.0,Main,3.0,1,50,"[(49.905674025615, -97.134404708765), (49.9062...",5
28959,18560.0,Digby,2.0,0,50,"[(49.913833165245, -97.125132844509), (49.9138...",34
28960,24835.0,Disraeli,3.0,2,60,"[(49.902763600349, -97.128799417643), (49.9028...",14


In [117]:
ON_TIME = import_data("ON_TIME")
ON_TIME

,Stop Number,Route Number,Day Type,Scheduled Time,Deviation,Long,Lat,Route,Delay Type
0,30893,71,Weekday,2021-08-03 12:57:00,-83,-97.148700,49.958300,"(71, Arlington, Portage via Sinclair)",on-time
1,30894,71,Weekday,2021-08-03 12:57:47,-131,-97.144732,49.957068,"(71, Arlington, Portage via Sinclair)",early
2,30884,71,Weekday,2021-08-03 12:58:20,-112,-97.145920,49.955548,"(71, Arlington, Portage via Sinclair)",on-time
3,30377,71,Weekday,2021-08-03 13:00:00,-128,-97.145247,49.952372,"(71, Arlington, Portage via Sinclair)",early
4,30378,71,Weekday,2021-08-03 13:01:02,-277,-97.140930,49.951178,"(71, Arlington, Portage via Sinclair)",early
...,...,...,...,...,...,...,...,...,...
4264663,40043,47,Weekday,2021-08-17 10:26:36,-252,-96.992926,49.895046,"(47, Transcona - Pembina, Transcona via Regent)",early
4264664,40037,47,Weekday,2021-08-17 10:27:13,-248,-96.989166,49.895062,"(47, Transcona - Pembina, Transcona via Regent)",early
4264665,40031,47,Weekday,2021-08-17 10:27:42,-248,-96.986778,49.895893,"(47, Transcona - Pembina, Transcona via Regent)",early
4264666,40029,47,Weekday,2021-08-17 10:28:03,-240,-96.984648,49.895901,"(47, Transcona - Pembina, Transcona via Regent)",early


In [3]:
TRAFFIC_COUNTS = import_data("TRAFFIC_COUNTS")
TRAFFIC_COUNTS

,Timestamp,Site,Northbound,Southbound,Eastbound,Westbound,Total,Lat,Long,Street,Near
0,2021-08-01 20:00:00,McPhillips And 190m South Of Leila,170.0,152.0,NaN,NaN,322,49.951733,-97.149032,McPhillips,Leila
1,2021-08-01 02:00:00,McPhillips And 190m South Of Leila,14.0,15.0,NaN,NaN,29,49.951733,-97.149032,McPhillips,Leila
2,2021-08-01 19:30:00,McPhillips And 190m South Of Leila,181.0,161.0,NaN,NaN,342,49.951733,-97.149032,McPhillips,Leila
3,2021-08-01 11:15:00,McPhillips And 190m South Of Leila,178.0,262.0,NaN,NaN,440,49.951733,-97.149032,McPhillips,Leila
4,2021-08-01 04:15:00,Henderson And 55M S Of Frasers Grove,12.0,12.0,NaN,NaN,24,49.934985,-97.096166,Henderson,Grove
...,...,...,...,...,...,...,...,...,...,...,...
21490,2021-08-24 14:00:00,Marion And 260M E Of Dupuy,NaN,NaN,180.0,198.0,378,49.881900,-97.089342,Marion,Dupuy
21491,2021-08-24 11:15:00,Lagimodiere And 80M N Of Burmac,249.0,282.0,NaN,NaN,531,49.848695,-97.049665,Lagimodiere,Burmac
21492,2021-08-24 22:00:00,Lagimodiere And 80M N Of Burmac,127.0,171.0,NaN,NaN,298,49.848695,-97.049665,Lagimodiere,Burmac
21493,2021-08-24 07:15:00,Pembina And 280 N Of Adamar,326.0,131.0,NaN,NaN,457,49.826952,-97.152312,Pembina,Adamar


In [4]:
SITES = import_data("SITES")
SITES

,Site,Street,Near,Lat,Long
0,McPhillips And 190m South Of Leila,McPhillips,Leila,49.951733,-97.149032
1,Henderson And 55M S Of Frasers Grove,Henderson,Grove,49.934985,-97.096166
2,Pembina And 280 N Of Adamar,Pembina,Adamar,49.826952,-97.152312
3,Inkster And 130W Of Wyatt,Inkster,Wyatt,49.945949,-97.187758
4,Nichol And St.Marys,Nichol,St.Marys,49.850579,-97.112202
5,Lagimodiere And 80M N Of Burmac,Lagimodiere,Burmac,49.848695,-97.049665
6,Disraeli Bridge,Disraeli,Bridge,49.906744,-97.123028
7,Marion And 260M E Of Dupuy,Marion,Dupuy,49.881900,-97.089342


In [5]:
STOPS = import_data("STOPS")
STOPS

,Stop Number,Stop Name,Lat,Long,Direction,Street,At
0,10001,Southbound Osborne at Mulvey,49.871261,-97.139518,Southbound,Osborne,Mulvey
1,10002,Southbound Osborne at Woodward,49.868819,-97.137553,Southbound,Osborne,Woodward
2,10003,Southbound Osborne at Brandon,49.867880,-97.136795,Southbound,Osborne,Brandon
3,10004,Southbound Osborne at Hethrington,49.866522,-97.135707,Southbound,Osborne,Hethrington
4,10005,Southbound Osborne at Morley,49.865164,-97.134604,Southbound,Osborne,Morley
...,...,...,...,...,...,...,...
5151,62018,Eastbound Parker at Beaumont,49.847806,-97.164596,Eastbound,Parker,Beaumont
5152,62021,Northbound Eaglewood at Longspur,49.780139,-97.196271,Northbound,Eaglewood,Longspur
5153,62022,Southbound Eaglewood at Longspur,49.780498,-97.196785,Southbound,Eaglewood,Longspur
5154,62023,Westbound Bison at Appleford,49.791002,-97.209359,Westbound,Bison,Appleford


In [6]:
LANE_CLOSURE = import_data("LANE_CLOSURE")
LANE_CLOSURE

,Street,At,Boundaries,Direction,Date Closed - From,Date Closed - To,Time Closed - From,Time Closed - To,Complete Closure,Lat,Long,Geometry,Num_Segments,Num_Points
0,Bryce,River,"[River Av, End]","[Eastbound, Westbound]",2021-08-16,2021-10-16,12:00 AM,12:00 AM,No,49.879699,-97.142781,"[[(49.879153816388, -97.142387643857), (49.880...",1,3
1,Lilac,Carter,"[Carter Av, Weatherdon Av]",[Southbound],2021-06-14,2021-10-29,12:00 AM,12:00 AM,No,49.862454,-97.151445,"[[(49.862119196838, -97.151173092203), (49.862...",1,5
2,Princess,Pacific,"[Pacific Av, Rupert Av]",[Southbound],2020-08-14,2021-10-16,12:00 AM,12:00 AM,No,49.901786,-97.139766,"[[(49.90158994935, -97.139915854251), (49.9019...",1,3
3,Henry,Austin,"[Austin St, End]",[Westbound],2021-05-10,2021-11-30,12:00 AM,11:00 PM,No,49.903340,-97.134152,"[[(49.903503694415, -97.13466284795), (49.9033...",1,4
4,Grandin,Tache,"[Tache Av, St Joseph St]",[Eastbound],2021-05-31,2022-01-13,12:00 AM,12:00 AM,No,49.896013,-97.126315,"[[(49.895678857718, -97.127616387474), (49.896...",1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,Sherbrook,Logan,"[Logan Av, William Av]",[Southbound],2021-09-20,2021-11-01,12:00 AM,12:00 AM,No,49.906584,-97.153705,"[[(49.904179261579, -97.155653416614), (49.904...",1,30
426,Gull Lake,Markham,"[Markham Rd, East Lake Dr]","[Northbound, Southbound]",2021-09-17,2021-10-08,12:00 AM,12:00 AM,No,49.809311,-97.158976,"[[(49.80730904568, -97.158914003228), (49.8081...",1,39
427,Wardlaw,Osborne,"[Osborne St, Scott St]",[Eastbound],2021-09-20,2021-10-15,12:00 AM,12:00 AM,No,49.877534,-97.142232,"[[(49.876961712509, -97.143938755153), (49.877...",1,5
428,Scotsborough,Beckinsale,"[Beckinsale By, Novavista Dr]",[Southbound],2021-09-23,2021-10-13,12:00 AM,12:00 AM,No,49.820905,-97.107388,"[[(49.820524519173, -97.107184511392), (49.820...",1,4
